In [1]:
import time

from deap import base
from deap import creator
from deap import tools
from deap import algorithms

from ipynb.fs.full.ga_algorithm import *
from ipynb.fs.full.ga_nonogram import getInfo

from ipynb.fs.full.bf_nonogram import Puzzle

In [2]:
nonogram_test_input = {
    "rows": [[3], [2, 1], [3, 2], [2, 2], [6], [1, 5], [6], [1], [2]],
    "cols": [[1, 2], [3, 1], [1, 5], [7, 1], [5], [3], [4], [3]],
    "n": 800,
    "m_indpb": 0.05
}

In [3]:
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

In [4]:
start = time.time()

toolbox = base.Toolbox()
toolbox.register("population", createIndividuals, nonogram_test_input, creator.Individual)

toolbox.register("evaluate", evalOneMax, nonogram=nonogram_test_input)
toolbox.register("mate", cxTwoPoints)
toolbox.register("mutate", mutShuffleIndexes, indpb=nonogram_test_input.get("m_indpb"), nonogram=nonogram_test_input)
toolbox.register("select", tools.selTournament, tournsize=3)

pop = toolbox.population(n=nonogram_test_input.get("n"))
hof = tools.HallOfFame(1)

stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", np.mean)
stats.register("min", np.min)
stats.register("max", np.max)

pop, log = eaSimpleWithElitism(pop, toolbox, cxpb=0.5, mutpb=0.2, ngen=100, stats=stats, halloffame=hof, verbose=True)

end = time.time()

rows, cols, row_blocks, col_blocks = getInfo(nonogram_test_input)

print("\nBest individual is: %s" % hof[0])
result = getNonogramFromIndividual(rows, col_blocks, hof[0])

print("\nTime: ", round(end - start, 3), "sec")

print("\nNonogram:\n")

for row in result:
    print(''.join(map(lambda s: chr(9608) if s else ' ', row)))

gen	nevals	avg   	min	max
0  	800   	8.3875	5  	9  
1  	483   	7.88875	3  	9  
2  	457   	7.4    	3  	9  
3  	488   	6.94875	3  	9  
4  	456   	6.65   	3  	9  
5  	473   	6.24125	3  	9  
6  	471   	5.825  	2  	9  
7  	502   	5.31125	2  	9  
8  	452   	4.60625	0  	9  

Best individual is: [[1, 2], [0, 2], [0, 0], [0, 0], [4], [4], [2], [2]]

Time:  0.656 sec

Nonogram:

 ███    
██ █    
 ███  ██
  ██  ██
  ██████
█ █████ 
██████  
    █   
   ██   


In [5]:
start = time.time()

puzzle = Puzzle(cols, rows)

while True:
    if not puzzle.solve_step():
        end = time.time()
        print("no next solution step")
        print("\nTime: ", round(end - start, 3), "sec")
        print("\nNonogram:\n")
        puzzle.print()
        break
        

no next solution step

Time:  0.001 sec

Nonogram:

 ███    
██ █    
 ███  ██
  ██  ██
  ██████
█ █████ 
██████  
    █   
   ██   
